<a href="https://colab.research.google.com/github/josephgattuso/python-for-finance/blob/master/Portfolio_Asset_Allocation_and_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Assets Allocation and Statistical Data Analysis


## PROJECT OVERVIEW AND ASSET TYPES

*   In this project, instead of analyzing individual stocks, we will create a function to perform portfolio analysis such as calculating portfolio returns, risk, and Sharpe ratio
*   A portfolio is a collection of financial investments such as stocks, fixed income securities (bonds), cash, mutual funds, and Exchange Traded Funds (ETFs)
*   Let's assume that you have £1M to investment in the market. How can you allocate these assets among a selected set of securities?  Should you invest them all in Apple? S&P500? Bonds?
*  The answer depends on many factors such as client's risk tolerance, target returns, and investment time span





Links:
- https://www.bankofcanada.ca/rates/interest-rates/canadian-bonds/
- https://ca.finance.yahoo.com/quote/AAPL?p=AAPL&.tsrc=fin-tre-srch
- https://investor.vanguard.com/etf/profile/performance/voo
- https://grow.acorns.com/warren-buffett-index-funds/



## IMPORT LIBRARIES & DATASETS AND PERFORM DATA VISUALIZATION

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [54]:
# Read the stock data file
stocks_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/stock.csv')
stocks_df

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.130000,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.350000,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.250000,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.730000,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.800000,194.449997,180.520004,26.760000,318.590851,1314.500000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,440.250000,174.279999,29.850000,16.719999,3205.030029,125.449997,1485.020020,1473.609985,3327.770020
2155,2020-08-06,455.609985,172.199997,29.840000,18.459999,3225.000000,126.120003,1489.579956,1500.099976,3349.159912
2156,2020-08-07,444.450012,170.020004,30.020000,19.030001,3167.459961,124.959999,1452.709961,1494.489990,3351.280029
2157,2020-08-10,450.910004,179.410004,30.200001,21.650000,3148.159912,127.110001,1418.569946,1496.099976,3360.469971


In [55]:
# Sort the data based on Date
stocks_df = stocks_df.sort_values(by = ['Date'])
stocks_df

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.130000,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.350000,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.250000,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.730000,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.800000,194.449997,180.520004,26.760000,318.590851,1314.500000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,440.250000,174.279999,29.850000,16.719999,3205.030029,125.449997,1485.020020,1473.609985,3327.770020
2155,2020-08-06,455.609985,172.199997,29.840000,18.459999,3225.000000,126.120003,1489.579956,1500.099976,3349.159912
2156,2020-08-07,444.450012,170.020004,30.020000,19.030001,3167.459961,124.959999,1452.709961,1494.489990,3351.280029
2157,2020-08-10,450.910004,179.410004,30.200001,21.650000,3148.159912,127.110001,1418.569946,1496.099976,3360.469971



*   Use Plotly express to visualize raw stock data and normalized ones


In [56]:
def normalize(df):

  x = df.copy()

  for i in x.columns[1:]:
    x[i] = x[i] / x[i][0]
    
  return x


In [57]:
def interactive_plot(df, title):

  fig = px.line(title = title)
  
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  
  fig.show()


In [58]:
# plot raw data
interactive_plot(stocks_df, "PRICES")

In [59]:
interactive_plot(normalize(stocks_df), 'NORMALIZED PRICES')

## ASSET ALLOCATION FRAMEWORKS

*   Asset allocation is an investment strategy that is used to allocated client's assets based on their risk tolerance, target returns, and investment time spance
*   The goal of portfolio managers is to maximize returns and reduce risks
*   There are many asset classes to choose from, such as equities, fixed income, cash and equivalents, real estate, commodities, etc.  

*  What is the common advice that financial advisors generally give to retired seniors when it comes to asset allocations?

In [60]:
# The conventional wisdom is to subtract client's age from 100 to calculate the % of portolio that should be allocated to stocks
# For Example: If you're 30 years old, you should have 70% allocated to stocks
# If you are 75 years old, you should invest 25% in stocks (old retired seniors generally recommend a more stable low risk portfolio). 

## RANDOM ASSET ALLOCATIONS AND PORTFOLIO DAILY RETURN

In [61]:
weights = np.array(np.random.random(9))
weights

array([0.29990287, 0.9211315 , 0.98804587, 0.39879403, 0.33254541,
       0.06262472, 0.54879578, 0.73724496, 0.47175367])

In [62]:
# Let's create random portfolio weights
# Portfolio weights must sum to 1 

# Set random seed
# np.random.seed(101)
np.random.seed()

# Create random weights for the stocks and normalize them
weights = np.array(np.random.random(9))

# Ensure that the sum of all weights are = 1
weights = weights / np.sum(weights) 
print(weights)



[0.08478861 0.10729148 0.20406597 0.24663364 0.08429411 0.08334323
 0.01013627 0.0855319  0.09391479]


In [63]:
# Normalize the stock avalues 
df_portfolio = normalize(stocks_df)
df_portfolio

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2012-01-13,0.996251,0.987949,0.998340,1.018137,1.014153,0.992301,0.806726,0.992615,0.995052
2,2012-01-17,1.007855,0.996424,1.004316,1.009893,1.032570,0.996954,0.941593,0.998317,0.998587
3,2012-01-18,1.018320,0.994040,1.006972,1.049464,1.076792,1.002880,0.949027,1.005193,1.009680
4,2012-01-19,1.015093,1.000662,1.009960,1.055235,1.105269,0.999834,0.947257,1.015771,1.014666
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,7.313297,2.308039,0.991036,1.378401,18.217644,0.694821,52.567080,4.698347,2.568715
2155,2020-08-06,7.568452,2.280493,0.990704,1.521847,18.331155,0.698532,52.728494,4.782805,2.585226
2156,2020-08-07,7.383066,2.251622,0.996680,1.568838,18.004093,0.692107,51.423361,4.764919,2.586862
2157,2020-08-10,7.490377,2.375977,1.002656,1.784831,17.894390,0.704016,50.214865,4.770052,2.593956


In [64]:
df_portfolio.columns[1:]

Index(['AAPL', 'BA', 'T', 'MGM', 'AMZN', 'IBM', 'TSLA', 'GOOG', 'sp500'], dtype='object')

In [65]:
weights

array([0.08478861, 0.10729148, 0.20406597, 0.24663364, 0.08429411,
       0.08334323, 0.01013627, 0.0855319 , 0.09391479])

In [66]:
# Note that enumerate returns the value and a counter as well
for counter, stock in enumerate(df_portfolio.columns[1:]):
  df_portfolio[stock] = df_portfolio[stock] * weights[counter]
  df_portfolio[stock] = df_portfolio[stock] * 1000000
df_portfolio

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,84788.607056,107291.482381,204065.967891,246633.640347,8.429411e+04,83343.225958,10136.273395,85531.902896,93914.786045
1,2012-01-13,84470.695749,105998.465886,203727.206200,251106.797880,8.548716e+04,82701.592068,8177.192241,84900.235814,93450.102933
2,2012-01-17,85454.620484,106907.836127,204946.723896,249073.544456,8.703956e+04,83089.340477,9544.243267,85387.911386,93782.126897
3,2012-01-18,86341.965602,106652.075747,205488.731761,258833.160892,9.076722e+04,83583.263677,9619.592198,85976.111175,94823.852104
4,2012-01-19,86068.315845,107362.521248,206098.490609,260256.438288,9.316769e+04,83329.378196,9601.652250,86880.822854,95292.154578
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,620084.235498,247632.882357,202236.684572,339959.952182,1.535640e+06,57908.652853,532834.298042,401858.520612,241240.300606
2155,2020-08-06,641718.499112,244677.426232,202168.933589,375338.561762,1.545208e+06,58217.932612,534470.431068,409082.432436,242790.919773
2156,2020-08-07,625999.877134,241579.893794,203388.451285,386928.146944,1.517639e+06,57682.466127,521241.250558,407552.569923,242944.613586
2157,2020-08-10,635098.660100,254922.060301,204607.975756,440199.366324,1.508392e+06,58674.923062,508991.603629,407991.618653,243610.820793


In [67]:
# Let's create an additional column that contains the sum of all $ values in the portfolio
df_portfolio['portfolio daily worth in $'] = df_portfolio[df_portfolio != 'Date'].sum(axis = 1)
df_portfolio

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $
0,2012-01-12,84788.607056,107291.482381,204065.967891,246633.640347,8.429411e+04,83343.225958,10136.273395,85531.902896,93914.786045,1.000000e+06
1,2012-01-13,84470.695749,105998.465886,203727.206200,251106.797880,8.548716e+04,82701.592068,8177.192241,84900.235814,93450.102933,1.000019e+06
2,2012-01-17,85454.620484,106907.836127,204946.723896,249073.544456,8.703956e+04,83089.340477,9544.243267,85387.911386,93782.126897,1.005226e+06
3,2012-01-18,86341.965602,106652.075747,205488.731761,258833.160892,9.076722e+04,83583.263677,9619.592198,85976.111175,94823.852104,1.022086e+06
4,2012-01-19,86068.315845,107362.521248,206098.490609,260256.438288,9.316769e+04,83329.378196,9601.652250,86880.822854,95292.154578,1.028057e+06
...,...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,620084.235498,247632.882357,202236.684572,339959.952182,1.535640e+06,57908.652853,532834.298042,401858.520612,241240.300606,4.179396e+06
2155,2020-08-06,641718.499112,244677.426232,202168.933589,375338.561762,1.545208e+06,58217.932612,534470.431068,409082.432436,242790.919773,4.253674e+06
2156,2020-08-07,625999.877134,241579.893794,203388.451285,386928.146944,1.517639e+06,57682.466127,521241.250558,407552.569923,242944.613586,4.204956e+06
2157,2020-08-10,635098.660100,254922.060301,204607.975756,440199.366324,1.508392e+06,58674.923062,508991.603629,407991.618653,243610.820793,4.262489e+06


In [68]:
# Let's calculate the portfolio daily return 
# Define a new column in the dataframe and set it to zeros
df_portfolio['portfolio daily % return'] = 0.0000

for i in range(1, len(stocks_df)):
  # Calculate the percentage of change from the previous day
  df_portfolio['portfolio daily % return'][i] = ( (df_portfolio['portfolio daily worth in $'][i] - df_portfolio['portfolio daily worth in $'][i-1]) / df_portfolio['portfolio daily worth in $'][i-1]) * 100 

df_portfolio

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $,portfolio daily % return
0,2012-01-12,84788.607056,107291.482381,204065.967891,246633.640347,8.429411e+04,83343.225958,10136.273395,85531.902896,93914.786045,1.000000e+06,0.000000
1,2012-01-13,84470.695749,105998.465886,203727.206200,251106.797880,8.548716e+04,82701.592068,8177.192241,84900.235814,93450.102933,1.000019e+06,0.001945
2,2012-01-17,85454.620484,106907.836127,204946.723896,249073.544456,8.703956e+04,83089.340477,9544.243267,85387.911386,93782.126897,1.005226e+06,0.520636
3,2012-01-18,86341.965602,106652.075747,205488.731761,258833.160892,9.076722e+04,83583.263677,9619.592198,85976.111175,94823.852104,1.022086e+06,1.677242
4,2012-01-19,86068.315845,107362.521248,206098.490609,260256.438288,9.316769e+04,83329.378196,9601.652250,86880.822854,95292.154578,1.028057e+06,0.584245
...,...,...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,620084.235498,247632.882357,202236.684572,339959.952182,1.535640e+06,57908.652853,532834.298042,401858.520612,241240.300606,4.179396e+06,1.185529
2155,2020-08-06,641718.499112,244677.426232,202168.933589,375338.561762,1.545208e+06,58217.932612,534470.431068,409082.432436,242790.919773,4.253674e+06,1.777240
2156,2020-08-07,625999.877134,241579.893794,203388.451285,386928.146944,1.517639e+06,57682.466127,521241.250558,407552.569923,242944.613586,4.204956e+06,-1.145299
2157,2020-08-10,635098.660100,254922.060301,204607.975756,440199.366324,1.508392e+06,58674.923062,508991.603629,407991.618653,243610.820793,4.262489e+06,1.368205


*  Try at least 3 another random weights allocation and rerun the code
*  Compare the final portfolio value on Aug 11th, 2020 to its initial value (£1M) on January 12th, 2012. Do you notice a big difference? Comment on your answer

In [70]:
# remove the seed value of 101, run the code 3 times, and record the final portolio value on the last day (Aug 11th, 2020)
np.random.seed()

# Portfolio Value Run #1 = $6.383645e+06, weights = [0.01246531 0.23749173 0.18257962 0.03419922 0.08369018 0.23405306
#  0.07120053 0.00824087 0.13607948]

# Portfolio Value Run #2 = $9.594104e+06, Weights = [0.13932903 0.04729773 0.15580758 0.03232255 0.0629491  0.15081605
#  0.12206726 0.16938746 0.12002324]

# Portfolio Value Run #3 = 1.138105e+07, Weights = [0.17707261 0.06977804 0.02394314 0.1550952  0.11152213 0.12017452
# 0.14296447 0.05729485 0.14215505]

## PORTFOLIO ALLOCATION - DAILY RETURN CALCULATION (FUNCTION)

In [75]:
# Lets assume we have $1,000,000 to be invested and we will allocate this fund based on the weights of the stocks
# We will create a function that takes in the stock prices along with the weights and return:
# (1) Daily value of each individual security in $ over the specified time period
# (2) Overall daily worth of the entire portfolio 
# (3) Daily return 

def portfolio_allocation(df, weights):

  df_portfolio = df.copy()
  
  # Normalize the stock avalues 
  df_portfolio = normalize(df_portfolio)
  
  for counter, stock in enumerate(df_portfolio.columns[1:]):
    df_portfolio[stock] = df_portfolio[stock] * weights[counter]
    df_portfolio[stock] = df_portfolio[stock] * 1000000

  df_portfolio['portfolio daily worth in $'] = df_portfolio[df_portfolio != 'Date'].sum(axis = 1)
  
  df_portfolio['portfolio daily % return'] = 0.0000

  for i in range(1, len(stocks_df)):
    
    # Calculate the percentage of change from the previous day
    df_portfolio['portfolio daily % return'][i] = ( (df_portfolio['portfolio daily worth in $'][i] - df_portfolio['portfolio daily worth in $'][i-1]) / df_portfolio['portfolio daily worth in $'][i-1]) * 100 
  
  # set the value of first row to zero, as previous value is not available
  df_portfolio['portfolio daily % return'][0] = 0
  return df_portfolio

*  Call the function and ensure that the results make sense

In [76]:
df_portfolio = portfolio_allocation(stocks_df, weights)
df_portfolio

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $,portfolio daily % return
0,2012-01-12,84788.607056,107291.482381,204065.967891,246633.640347,8.429411e+04,83343.225958,10136.273395,85531.902896,93914.786045,1.000000e+06,0.000000
1,2012-01-13,84470.695749,105998.465886,203727.206200,251106.797880,8.548716e+04,82701.592068,8177.192241,84900.235814,93450.102933,1.000019e+06,0.001945
2,2012-01-17,85454.620484,106907.836127,204946.723896,249073.544456,8.703956e+04,83089.340477,9544.243267,85387.911386,93782.126897,1.005226e+06,0.520636
3,2012-01-18,86341.965602,106652.075747,205488.731761,258833.160892,9.076722e+04,83583.263677,9619.592198,85976.111175,94823.852104,1.022086e+06,1.677242
4,2012-01-19,86068.315845,107362.521248,206098.490609,260256.438288,9.316769e+04,83329.378196,9601.652250,86880.822854,95292.154578,1.028057e+06,0.584245
...,...,...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,620084.235498,247632.882357,202236.684572,339959.952182,1.535640e+06,57908.652853,532834.298042,401858.520612,241240.300606,4.179396e+06,1.185529
2155,2020-08-06,641718.499112,244677.426232,202168.933589,375338.561762,1.545208e+06,58217.932612,534470.431068,409082.432436,242790.919773,4.253674e+06,1.777240
2156,2020-08-07,625999.877134,241579.893794,203388.451285,386928.146944,1.517639e+06,57682.466127,521241.250558,407552.569923,242944.613586,4.204956e+06,-1.145299
2157,2020-08-10,635098.660100,254922.060301,204607.975756,440199.366324,1.508392e+06,58674.923062,508991.603629,407991.618653,243610.820793,4.262489e+06,1.368205


## PORTFOLIO DATA VISUALIZATION

In [78]:
# Plot the portfolio daily % return
fig = px.line(x = df_portfolio['Date'], y = df_portfolio['portfolio daily % return'], title = 'PORTFOLIO DAILY % RETURN')
fig.show()

In [80]:
# Plot all stocks (normalized)

interactive_plot(df_portfolio.drop(['portfolio daily worth in $', 'portfolio daily % return'], axis = 1), 'Portfolio individual stocks worth in $ over time')

In [81]:
# Print out a histogram of daily returns
fig = px.histogram(df_portfolio, x = 'portfolio daily % return')
fig.show()

*   Plot the portfolio overall daily worth vs. time
*   Rerun the code with various weights and visualize the final value

In [82]:
fig = px.line(x = df_portfolio.Date, y = df_portfolio['portfolio daily worth in $'], title= 'Portfolio Overall Value in $')
fig.show()

## PORTFOLIO STATISTICAL METRICS (CUMULATIVE RETURN, AVERAGE DAILY RETURN, AND SHARPE RATIO)

### Returns

*   Stock daily return is a calculation of how much invetsors have gained or lost per day
*   Cumulative return is a measure of the aggregate amount that the stock gained or lost over a period of time



### Standard Deviation


*   Standard deviation is a measurement of the dispersion away from the mean
*   The more spread the data, the higher the standard deviation
*   Volatile stocks have high standard deviation and therefore standard deviation represent the risk associated with the security





### Sharpe Ratio

*   Sharpe Ratio is used by investors to calculate the return of an investment compared to its risk
*   Sharpe Ratio is simply a calculation of the average return earned in excess of the risk-free rate (ex: US Government bonds) per unit of risk (volatility or standard deviation of the portfolio)
*   As Sharpe Ratio increases, risk-adjusted return increases and security becomes more desired by investors

## PORTFOLIO STATISTICAL METRICS (CUMULATIVE RETURN, AVERAGE DAILY RETURN, AND SHARPE RATIO)

In [83]:
df_portfolio

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $,portfolio daily % return
0,2012-01-12,84788.607056,107291.482381,204065.967891,246633.640347,8.429411e+04,83343.225958,10136.273395,85531.902896,93914.786045,1.000000e+06,0.000000
1,2012-01-13,84470.695749,105998.465886,203727.206200,251106.797880,8.548716e+04,82701.592068,8177.192241,84900.235814,93450.102933,1.000019e+06,0.001945
2,2012-01-17,85454.620484,106907.836127,204946.723896,249073.544456,8.703956e+04,83089.340477,9544.243267,85387.911386,93782.126897,1.005226e+06,0.520636
3,2012-01-18,86341.965602,106652.075747,205488.731761,258833.160892,9.076722e+04,83583.263677,9619.592198,85976.111175,94823.852104,1.022086e+06,1.677242
4,2012-01-19,86068.315845,107362.521248,206098.490609,260256.438288,9.316769e+04,83329.378196,9601.652250,86880.822854,95292.154578,1.028057e+06,0.584245
...,...,...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,620084.235498,247632.882357,202236.684572,339959.952182,1.535640e+06,57908.652853,532834.298042,401858.520612,241240.300606,4.179396e+06,1.185529
2155,2020-08-06,641718.499112,244677.426232,202168.933589,375338.561762,1.545208e+06,58217.932612,534470.431068,409082.432436,242790.919773,4.253674e+06,1.777240
2156,2020-08-07,625999.877134,241579.893794,203388.451285,386928.146944,1.517639e+06,57682.466127,521241.250558,407552.569923,242944.613586,4.204956e+06,-1.145299
2157,2020-08-10,635098.660100,254922.060301,204607.975756,440199.366324,1.508392e+06,58674.923062,508991.603629,407991.618653,243610.820793,4.262489e+06,1.368205


In [84]:
# Cummulative return of the portfolio (Note that we now look for the last net worth of the portfolio compared to it's start value)
cummulative_return = ((df_portfolio['portfolio daily worth in $'][-1:] - df_portfolio['portfolio daily worth in $'][0])/ df_portfolio['portfolio daily worth in $'][0]) * 100
print('Cummulative return of the portfolio is {} %'.format(cummulative_return.values[0]))


Cummulative return of the portfolio is 318.6974572700812 %


In [85]:
# Calculate the portfolio standard deviation
print('Standard deviation of the portfolio is {}'.format(df_portfolio['portfolio daily % return'].std()))

Standard deviation of the portfolio is 1.3178397824586676


In [86]:
# Calculate the average daily return 
print('Average daily return of the portfolio is {} %'.format(df_portfolio['portfolio daily % return'].mean() ))

Average daily return of the portfolio is 0.07509321427071453 %


In [87]:
# Portfolio sharpe ratio
sharpe_ratio = df_portfolio['portfolio daily % return'].mean() / df_portfolio['portfolio daily % return'].std() * np.sqrt(252)
print('Sharpe ratio of the portfolio is {}'.format(sharpe_ratio))

Sharpe ratio of the portfolio is 0.9045620237907569


*  Try at least 3 different random weights allocation, rerun the code and compare sharpe ratios, daily return and cummulative returns

In [88]:
# Rerun the code with 3 random values and compare metrics